# The goal of this notebook is to fix issues with duplicated reactions and other incositencies

In [1]:
%matplotlib inline
import os
import sys
sys.path.append('/home/sergio/Dropbox/s/cthermgem-dev')
sys.path.append('C:\\Users\\sergio\\Dropbox\\s\\cthermgem-dev\\')

import cobra as cb
import tools.ms2bigg
import settings

In [2]:
model_ms = cb.io.read_sbml_model(os.path.join(settings.PROJECT_ROOT,'iCBI','iCBI665_v3.sbml'))
model = tools.ms2bigg.main(model_ms)

# Consolidate reactions present in separately in both directions
I spotted the case of GRTT and FRTT, the same reaction  but in different directions, coded by the same gene. To avoid TICs this should be a reversible reaction.
Let's do the analysis in a systematic way


In [3]:
same_mets = []
for rxn_1 in model.reactions:
    met_ids_1 = [met.id for met in rxn_1.metabolites.keys()]
    for rxn_2 in model.reactions:
        met_ids_2 = [met.id for met in rxn_2.metabolites.keys()]
        if rxn_1.id != rxn_2.id:
            if set(met_ids_1) == set(met_ids_2):
                pair = {rxn_1.id, rxn_2.id}
                if pair not in same_mets:
                    same_mets.append(pair)
                    print(pair)


{'PGMT', 'PGCM'}
{'FRTT', 'GRTT'}
{'ATPM', 'NA1abc'}
{'FUCabc', 'ATPM'}
{'ATPM', 'GLCabc'}
{'ATPM', 'SBTabc'}
{'ATPM', 'FRUabc'}
{'FUCabc', 'NA1abc'}
{'GLCabc', 'NA1abc'}
{'SBTabc', 'NA1abc'}
{'FRUabc', 'NA1abc'}
{'FUCabc', 'GLCabc'}
{'FUCabc', 'SBTabc'}
{'FUCabc', 'FRUabc'}
{'SBTabc', 'GLCabc'}
{'FRUabc', 'GLCabc'}
{'FRUabc', 'SBTabc'}
{'BIOMASS_CELLOBIOSE', 'BIOMASS_CELLULOSE'}
{'BIOMASS_NO_CELLULOSOME', 'BIOMASS_CELLULOSE'}
{'BIOMASS_CELLOBIOSE', 'BIOMASS_NO_CELLULOSOME'}


- PGMT-PGMC and FRTT-GRTT seem to be actual cases of duplication.
- ATPM and abc transporter point at an issue with abc trasnporters lacking the metabolite they should translocate
- BIOMASS reactions are "false positives" since they are expected to have the same metabolties.

In [4]:
isg = cb.io.load_json_model(os.path.join(settings.PROJECT_ROOT,'iSG676','iSG676_cb.json'))

In [5]:
model.reactions.GLCabc.reaction

'atp_c + h2o_c --> adp_c + h_c + pi_c'

In [6]:
isg.reactions.GLCabc.reaction

'atp_c + glc__D_e + h2o_c --> adp_c + glc__D_c + h_c + pi_c'

The issue with sugar ABC transporters was introduced somwhere along the conversion from iSG to iCBI

# Fix real duplicates

In [7]:
model.reactions.PGMT

Reaction identifier,PGMT
Name,alpha_D_Glucose_1_phosphate_1_6_phosphomutase_c0
Memory address,0x0273b7175fd0
Stoichiometry,g1p_c <=> g6p_c Glucose_1_phosphate_c0 <=> D_glucose_6_phosphate_c0
GPR,CLO1313_RS05070
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
model.reactions.PGCM

Reaction identifier,PGCM
Name,alpha_D_Glucose_1_phosphate_1_6_phosphomutase_c0
Memory address,0x0273b6dfd358
Stoichiometry,g1p_c <=> g6p_c Glucose_1_phosphate_c0 <=> D_glucose_6_phosphate_c0
GPR,CLO1313_RS05070
Lower bound,-1000.0
Upper bound,1000.0


Both reactions are the same, PGCM is less common

In [9]:
model.reactions.PGCM.delete()

In [10]:
model.reactions.GRTT

Reaction identifier,GRTT
Name,CustomReaction_c0
Memory address,0x0273b6eee9e8
Stoichiometry,ggdp_c + ipdp_c --> frdp_c + h_c + ppi_c Geranylgeranyl_diphosphate_c0 + Isopentenyldiphosphate_c0 --> Farnesyldiphosphate_c0 + H_plus__c0 + PPi_c0
GPR,CLO1313_RS07040
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.reactions.FRTT

Reaction identifier,FRTT
Name,trans_trans_Farnesyl_diphosphate_isopentenyl_diphosphate_farnesyltranstransferase_c0
Memory address,0x0273b6efdf98
Stoichiometry,frdp_c + ipdp_c --> ggdp_c + h_c + ppi_c Farnesyldiphosphate_c0 + Isopentenyldiphosphate_c0 --> Geranylgeranyl_diphosphate_c0 + H_plus__c0 + PPi_c0
GPR,CLO1313_RS07040
Lower bound,0.0
Upper bound,1000.0


These correspond to the same reaction in two different directions, consolidate into one reversible reaction.

In [12]:
model.reactions.GRTT.delete()
model.reactions.FRTT.bounds = (-1000,1000)

# Fix transporters

Based on the above analysis, the following need to be fixed:
- NA1abc
- FUCabc
- GLCabc
- SBTabc
- FRUabc
The name of these reactions was also changed to ATP_phosphohydrolase_c0, so the error might have been introduced by KBase
Additionally, the metabolites involved had been added exchange reactions that directly interact with the cytosolic species. These need to be removed:

In [13]:
model.metabolites.na1_c.reactions

frozenset({<Reaction PPAna at 0x273b89c0898>,
           <Reaction EX_na1_e_e0 at 0x273b89e25f8>})

In [14]:
model.reactions.EX_na1_e_e0

Reaction identifier,EX_na1_e_e0
Name,CustomReaction_e0
Memory address,0x0273b89e25f8
Stoichiometry,na1_c <=> Na_plus__c0 <=>
GPR,U
Lower bound,-1000.0
Upper bound,1000.0


Additionally, some are reversible and some are irreversible, this will not be modified now. 

### NA1abc

In [15]:
model.reactions.EX_na1_e_e0.delete()
# sodium exchange already exsists and thus is not added
rxn_id = 'NA1abc'
mets = {}
mets[model.metabolites.na1_c] = 1
mets[model.metabolites.na1_e] = -1
model.reactions.get_by_id(rxn_id).add_metabolites(mets) 
model.reactions.get_by_id(rxn_id).name = 'Sodium transport via ABC system'
model.reactions.get_by_id(rxn_id).reaction


'atp_c + h2o_c + na1_e --> adp_c + h_c + na1_c + pi_c'

### FUCabc


In [16]:
internal = model.metabolites.fuc__L_c
internal

Metabolite identifier,fuc__L_c
Name,L_Fucose_c0
Memory address,0x0273b6c03978
Formula,None
Compartment,c0
In 2 reaction(s),"FCLK, EX_fuc__L_e"


In [17]:
external = cb.Metabolite(id='fuc__L_e',formula=internal.formula,name=internal.name,compartment='e0',charge=internal.charge)
model.add_metabolites(external)

In [18]:
model.reactions.EX_fuc__L_e.delete()
ex = model.add_boundary(external, lb=0) # lb not working here
ex.lower_bound = 0
rxn_id = 'FUCabc'
mets = {}
mets[model.metabolites.fuc__L_c] = 1
mets[model.metabolites.fuc__L_e] = -1

model.reactions.get_by_id(rxn_id).add_metabolites(mets) 
model.reactions.get_by_id(rxn_id).name = 'Fucose transport via ABC system'
model.reactions.get_by_id(rxn_id).reaction


'atp_c + fuc__L_e + h2o_c --> adp_c + fuc__L_c + h_c + pi_c'

### GLCabc

In [19]:
internal = model.metabolites.glc__D_c
internal

Metabolite identifier,glc__D_c
Name,D_Glucose_c0
Memory address,0x0273b6a157f0
Formula,None
Compartment,c0
In 9 reaction(s),"CLBH, KLCGH, CEPA, LACZ, ABGPT, EX_glc__D_e, HEX1, BDG2HCGHD, CGGH"


In [20]:
external = cb.Metabolite(id='glc__D_e',formula=internal.formula,name=internal.name,compartment='e0',charge=internal.charge)
model.add_metabolites(external)

In [21]:
model.reactions.EX_glc__D_e.delete()
ex = model.add_boundary(external, lb=0)
ex.lower_bound = 0

rxn_id = 'GLCabc'
mets = {}
mets[model.metabolites.glc__D_c] = 1
mets[model.metabolites.glc__D_e] = -1

model.reactions.get_by_id(rxn_id).add_metabolites(mets) 
model.reactions.get_by_id(rxn_id).name = 'Glucose transport via ABC system'
model.reactions.get_by_id(rxn_id).reaction

'atp_c + glc__D_e + h2o_c --> adp_c + glc__D_c + h_c + pi_c'

### SBTabc

In [22]:
internal = model.metabolites.sbt__D_c
internal

Metabolite identifier,sbt__D_c
Name,Sorbitol_c0
Memory address,0x0273b6a38be0
Formula,None
Compartment,c0
In 2 reaction(s),"SBTD_D2, Ex_sbt__D_e"


In [23]:
external = cb.Metabolite(id='sbt__D_e',formula=internal.formula,name=internal.name,compartment='e0',charge=internal.charge)
model.add_metabolites(external)

In [24]:
model.reactions.Ex_sbt__D_e.delete()
ex = model.add_boundary(external, lb=0)
ex.lower_bound = 0

rxn_id = 'SBTabc'
mets = {}
mets[model.metabolites.sbt__D_c] = 1
mets[model.metabolites.sbt__D_e] = -1

model.reactions.get_by_id(rxn_id).add_metabolites(mets) 
model.reactions.get_by_id(rxn_id).name = 'Sorbitol transport via ABC system'
model.reactions.get_by_id(rxn_id).reaction

'atp_c + h2o_c + sbt__D_e --> adp_c + h_c + pi_c + sbt__D_c'

### FRUabc

In [25]:
internal = model.metabolites.fru_c
internal

Metabolite identifier,fru_c
Name,D_Fructose_c0
Memory address,0x0273b6a38b38
Formula,None
Compartment,c0
In 3 reaction(s),"EX_fru_e, SBTD_D2, HEX7"


In [26]:
external = cb.Metabolite(id='fru_e',formula=internal.formula,name=internal.name,compartment='e0',charge=internal.charge)
model.add_metabolites(external)

In [27]:
model.reactions.EX_fru_e.delete()
ex = model.add_boundary(external, lb=0)
ex.lower_bound = 0

rxn_id = 'FRUabc'
mets = {}
mets[model.metabolites.fru_c] = 1
mets[model.metabolites.fru_e] = -1

model.reactions.get_by_id(rxn_id).add_metabolites(mets) 
model.reactions.get_by_id(rxn_id).name = 'Fructose transport via ABC system'
model.reactions.get_by_id(rxn_id).reaction

'atp_c + fru_e + h2o_c --> adp_c + fru_c + h_c + pi_c'

# Test if any of the new uptake reactions are blocked:


In [28]:
ex_map = {'NA1abc':'EX_na1_e', 'FUCabc':'EX_fuc__L_e','GLCabc':'EX_glc__D_e', 'SBTabc':'EX_sbt__D_e', 'FRUabc':'EX_fru_e'}

with model as tmodel:
    print('trans_id \t transporter_bounds  \t max_flux \t ex_id \t\t original_ex_bounds \t new_ex_bounds')
    for rxn_id in ['NA1abc', 'FUCabc','GLCabc', 'SBTabc', 'FRUabc']:
        rxn = tmodel.reactions.get_by_id(rxn_id)
        transporter_bounds = rxn.bounds
        ex_rxn = tmodel.reactions.get_by_id(ex_map[rxn_id])
        original_exchange_bounds = ex_rxn.bounds
        ex_rxn.lower_bound = -1000
        
        tmodel.objective = rxn
        r = tmodel.optimize(objective_sense='maximize')
        print('{} \t \t {} \t \t {:.2f} \t {} \t {} \t\t {}'.format(rxn.id, rxn.bounds, r.objective_value, ex_rxn.id, original_exchange_bounds, ex_rxn.bounds))

trans_id 	 transporter_bounds  	 max_flux 	 ex_id 		 original_ex_bounds 	 new_ex_bounds
NA1abc 	 	 (0.0, 1000.0) 	 	 1000.00 	 EX_na1_e 	 (-1000.0, 1000.0) 		 (-1000.0, 1000.0)
FUCabc 	 	 (0.0, 1000.0) 	 	 1000.00 	 EX_fuc__L_e 	 (0, 1000.0) 		 (-1000, 1000.0)
GLCabc 	 	 (0.0, 1000.0) 	 	 1000.00 	 EX_glc__D_e 	 (0, 1000.0) 		 (-1000, 1000.0)
SBTabc 	 	 (0.0, 1000.0) 	 	 1000.00 	 EX_sbt__D_e 	 (0, 1000.0) 		 (-1000, 1000.0)
FRUabc 	 	 (0.0, 1000.0) 	 	 1000.00 	 EX_fru_e 	 (0, 1000.0) 		 (-1000, 1000.0)


All transporter may carry flux if the exchanges (closed by default except for sodium) are opened. 

# Save updated model

In [31]:
cb.io.write_sbml_model(model,'iCBI665_v4_bigg.sbml')